# GeekBrains, Recommendation Systems
# Lesson 5 Homework

**Импорт библиотек**

In [ ]:
import itertools as it

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

import optuna

# Функции из 1-ого вебинара
# from src.metrics import precision_at_k, recall_at_k
from utils import prefilter_items


%matplotlib inline

## Задание 1
Прочитать статьи про BPR, WARP loss:
- BPR loss - https://towardsdatascience.com/recommender-system-using-bayesian-personalized-ranking-d30e98bba0b9
- WARP loss - https://medium.com/@gabrieltseng/intro-to-warp-loss-automatic-differentiation-and-pytorch-b6aa5083187a

### Решение Задания 1

+

## Задание 2
Сделать GridSearch текущей модели.

### Решение Задания 2

In [42]:
data = pd.read_csv('../data/retail_train.csv')

item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

## 1. Filter items

In [3]:
# n_items_before = data_train['item_id'].nunique()

# data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

# n_items_after = data_train_filtered['item_id'].nunique()
# print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

D:\Programming\GeekBrains\Term4\2 - GB_Rec_Systems_2021_05_31\homework\lesson5\utils.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices['price'] = data['sales_value'] / data['quantity']


Decreased # items from 86865 to 5001


# 2. Prepare data set

## 2.1 Prepare CSR train matrix

In [43]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)

## 2.2 Prepare CSR test matrix

In [44]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

test_sparse_user_item = csr_matrix(test_user_item_matrix)

## 2.3 Prepare ID dicts

In [45]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [46]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
user_feat.shape

(2499, 7)

In [48]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [49]:
item_feat.shape

(86865, 6)

## Encoding features

In [50]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [51]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Init model

In [56]:
scores = []

In [57]:
def objective(trial):
    global scores

    # Init
    loss = 'warp'
    components = trial.suggest_int('components', 16, 64)
    lr = trial.suggest_uniform('lr', 0.0001, 0.5)
    item_alpha = 0.1
    user_alpha = 0.1
    model = LightFM(no_components=components,
    #                 loss='bpr',
                    loss=loss,
                    learning_rate=lr, 
                    item_alpha=item_alpha,
                    user_alpha=user_alpha, 
                    random_state=42)

    # Train
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values),
              item_features=csr_matrix(item_feat_lightfm.values),
              epochs=15, 
              num_threads=6,
              verbose=False)

    # Train evaluation
    train_precision = precision_at_k(model, sparse_user_item, 
                                     user_features=csr_matrix(user_feat_lightfm.values),
                                     item_features=csr_matrix(item_feat_lightfm.values),
                                     k=5).mean()
    # Test evaluation
    test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix), 
                                     user_features=csr_matrix(user_feat_lightfm.values),
                                     item_features=csr_matrix(item_feat_lightfm.values),
                                     k=5).mean()

    # Save scores
    scores.append({'components': components,
                   'lr': lr,
                   'train': train_precision,
                   'test': test_precision})

    return test_precision


study = optuna.create_study(direction='maximize')  # Create a new study.
study.optimize(objective, n_trials=25)  # Invoke optimization of the objective function.

[I 2021-06-21 21:27:29,862] A new study created in memory with name: no-name-dce18a51-8c81-4f1d-a8bd-2ec8a1b9a71f
[I 2021-06-21 21:40:08,609] Trial 0 finished with value: 0.0023517883382737637 and parameters: {'components': 57, 'lr': 0.15913248234726596}. Best is trial 0 with value: 0.0023517883382737637.
[I 2021-06-21 21:50:12,540] Trial 1 finished with value: 0.000979911768808961 and parameters: {'components': 63, 'lr': 0.43951364743156746}. Best is trial 0 with value: 0.0023517883382737637.
[I 2021-06-21 21:59:48,874] Trial 2 finished with value: 0.003821656107902527 and parameters: {'components': 52, 'lr': 0.18044347464537877}. Best is trial 2 with value: 0.003821656107902527.
[I 2021-06-21 22:07:00,976] Trial 3 finished with value: 0.0 and parameters: {'components': 37, 'lr': 0.25430400220412014}. Best is trial 2 with value: 0.003821656107902527.
[I 2021-06-21 22:51:49,355] Trial 4 finished with value: 0.005291524343192577 and parameters: {'components': 38, 'lr': 0.129765972506919

Лучшая модель имеет параметры no_components = 60, learning_rate = 0.149.